In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [2]:
flights_df = pd.read_csv('Atlanta_01_2023_Flights.csv')

In [3]:
flights_df

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,MKT_UNIQUE_CARRIER,MKT_CARRIER_FL_NUM,ORIGIN_CITY_NAME,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,DEP_DELAY_GROUP,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,1,1,7,AA,1006,"Miami, FL","Atlanta, GA",2100,2058.0,0.0,-1.0,3,NaN,NaN,NaN,NaN,NaN
1,1,1,7,AA,1045,"Miami, FL","Atlanta, GA",1925,1922.0,0.0,-1.0,3,NaN,NaN,NaN,NaN,NaN
2,1,1,7,AA,1051,"Atlanta, GA","Dallas/Fort Worth, TX",1817,1816.0,0.0,-1.0,3,NaN,NaN,NaN,NaN,NaN
3,1,1,7,AA,1051,"Dallas/Fort Worth, TX","Atlanta, GA",1425,1420.0,0.0,-1.0,3,NaN,NaN,NaN,NaN,NaN
4,1,1,7,AA,1151,"Atlanta, GA","Miami, FL",602,553.0,0.0,-1.0,3,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56164,1,31,2,WN,972,"Atlanta, GA","Cleveland, OH",1635,1631.0,0.0,-1.0,3,NaN,NaN,NaN,NaN,NaN
56165,1,31,2,WN,972,"Denver, CO","Atlanta, GA",1100,1107.0,7.0,0.0,5,NaN,NaN,NaN,NaN,NaN
56166,1,31,2,WN,993,"Atlanta, GA","Omaha, NE",900,856.0,0.0,-1.0,4,NaN,NaN,NaN,NaN,NaN
56167,1,31,2,WN,993,"Fort Lauderdale, FL","Atlanta, GA",615,613.0,0.0,-1.0,3,NaN,NaN,NaN,NaN,NaN


In [4]:
flights_df['MKT_UNIQUE_CARRIER'].unique()

array(['AA', 'AS', 'B6', 'DL', 'F9', 'NK', 'UA', 'WN', 'G4'], dtype=object)

In [5]:
#Only look at flights leaving Atlanta on Delta
DL_flights_df = flights_df.loc[(flights_df['MKT_UNIQUE_CARRIER']=='DL')&(flights_df['ORIGIN_CITY_NAME']=='Atlanta, GA')]
#Seperate the data into features and targets.
X=DL_flights_df[['DAY_OF_MONTH','DAY_OF_WEEK','DEST_CITY_NAME','CRS_DEP_TIME','DISTANCE_GROUP']]
X

,DAY_OF_MONTH,DAY_OF_WEEK,DEST_CITY_NAME,CRS_DEP_TIME,DISTANCE_GROUP
111,1,7,"Dallas/Fort Worth, TX",2230,3
113,1,7,"Wichita, KS",1105,4
116,1,7,"New Orleans, LA",1359,2
118,1,7,"Tallahassee, FL",1515,1
120,1,7,"Portland, OR",1755,9
...,...,...,...,...,...
55767,31,2,"Los Angeles, CA",2024,8
55769,31,2,"Daytona Beach, FL",2235,2
55774,31,2,"Seattle, WA",810,9
55775,31,2,"Ontario, CA",2045,8


In [6]:
#Turn DEST_CITY_NAME into numeric values
dest_cities = X['DEST_CITY_NAME'].unique().tolist()
dest_cities_ids = []
for i in X.iterrows():
    dest_cities_ids.append(dest_cities.index(i[1]['DEST_CITY_NAME']))
X['Dest_City_id']=dest_cities_ids

C:\Users\Matt\AppData\Local\Temp\ipykernel_12044\3652359326.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Dest_City_id']=dest_cities_ids


In [7]:
#Drop the original DEST_CITY_NAME column
X=X.drop('DEST_CITY_NAME',axis=1)
X

,DAY_OF_MONTH,DAY_OF_WEEK,CRS_DEP_TIME,DISTANCE_GROUP,Dest_City_id
111,1,7,2230,3,0
113,1,7,1105,4,1
116,1,7,1359,2,2
118,1,7,1515,1,3
120,1,7,1755,9,4
...,...,...,...,...,...
55767,31,2,2024,8,12
55769,31,2,2235,2,112
55774,31,2,810,9,88
55775,31,2,2045,8,14


In [8]:
#Scale the feature data, X
from sklearn.preprocessing import StandardScaler
# Scaling the numeric columns
scaler = StandardScaler()
scaler.fit(X)
transformed_data = scaler.transform(X)
# Creating a DataFrame with with the scaled data
transformed_data_df = pd.DataFrame(transformed_data)

transformed_data_df

,0,1,2,3,4
0,-1.718081,1.508070,1.465448,-0.088702,-1.530221
1,-1.718081,1.508070,-0.789123,0.406400,-1.503909
2,-1.718081,1.508070,-0.280091,-0.583804,-1.477597
3,-1.718081,1.508070,0.032543,-1.078906,-1.451285
4,-1.718081,1.508070,0.513518,2.881911,-1.424973
...,...,...,...,...,...
19927,1.651297,-0.903545,1.052611,2.386809,-1.214477
19928,1.651297,-0.903545,1.475469,-0.583804,1.416725
19929,1.651297,-0.903545,-1.380322,2.881911,0.785237
19930,1.651297,-0.903545,1.094697,2.386809,-1.161853


In [9]:
from sklearn.decomposition import PCA
# Create a PCA model
pca = PCA(n_components=3)

In [10]:
# Use the PCA model with `fit_transform` to reduce to
# three principal components.
pca_data = pca.fit_transform(transformed_data_df)

pca_dict = {}
for i in range(0,3):
    pca_dict[f'pca{i+1}']=pca_data[:,i]
# Display sample data
pca_df = pd.DataFrame(pca_dict)
pca_df

,pca1,pca2,pca3
0,0.505913,0.084625,2.904195
1,1.670640,0.041999,1.822723
2,0.782476,0.067469,2.029802
3,0.308639,0.081514,2.159592
4,2.842662,0.022838,2.534571
...,...,...,...
19927,1.851107,-0.557332,-0.777355
19928,-1.974992,-0.419412,-1.079311
19929,1.866103,-0.541895,-2.244402
19930,1.803000,-0.555055,-0.764874


In [11]:
#Seperate the data into features and targets
is_delayed=DL_flights_df['DEP_DELAY_NEW'].tolist()
y=[]
#Determine if the flight was delayed using the DEP_DELAY_NEW column and
#convert to boolean values
for i in is_delayed:
    if i == 0:
        y.append(0)
    else:
        y.append(1)
y = pd.DataFrame(y)
y

,0
0,0
1,0
2,0
3,1
4,0
...,...
19927,1
19928,0
19929,1
19930,1


In [12]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(transformed_data_df, y, random_state=78)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=6, activation="relu", input_dim=5))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=6, activation="relu"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=6, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 36        
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 6)                 42        
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 127 (508.00 Byte)
Trainable params: 127 (508.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=100)

Epoch 1/100


468/468 [==============================] - 1s 683us/step - loss: 0.6626 - accuracy: 0.6095
Epoch 2/100
468/468 [==============================] - 0s 679us/step - loss: 0.6500 - accuracy: 0.6307
Epoch 3/100
468/468 [==============================] - 0s 608us/step - loss: 0.6456 - accuracy: 0.6357
Epoch 4/100
468/468 [==============================] - 0s 604us/step - loss: 0.6421 - accuracy: 0.6411
Epoch 5/100
468/468 [==============================] - 0s 600us/step - loss: 0.6384 - accuracy: 0.6475
Epoch 6/100
468/468 [==============================] - 0s 600us/step - loss: 0.6343 - accuracy: 0.6506
Epoch 7/100
468/468 [==============================] - 0s 602us/step - loss: 0.6307 - accuracy: 0.6535
Epoch 8/100
468/468 [==============================] - 0s 600us/step - loss: 0.6279 - accuracy: 0.6575
Epoch 9/100
468/468 [==============================] - 0s 600us/step - loss: 0.6252 - accuracy: 0.6597
Epoch 10/100
468/468 [==============================] - 0s 598us/step -

468/468 [==============================] - 0s 600us/step - loss: 0.6063 - accuracy: 0.6800
Epoch 76/100
468/468 [==============================] - 0s 604us/step - loss: 0.6062 - accuracy: 0.6798
Epoch 77/100
468/468 [==============================] - 0s 600us/step - loss: 0.6060 - accuracy: 0.6796
Epoch 78/100
468/468 [==============================] - 0s 603us/step - loss: 0.6062 - accuracy: 0.6792
Epoch 79/100
468/468 [==============================] - 0s 599us/step - loss: 0.6060 - accuracy: 0.6800
Epoch 80/100
468/468 [==============================] - 0s 604us/step - loss: 0.6060 - accuracy: 0.6812
Epoch 81/100
468/468 [==============================] - 0s 597us/step - loss: 0.6061 - accuracy: 0.6798
Epoch 82/100
468/468 [==============================] - 0s 606us/step - loss: 0.6056 - accuracy: 0.6801
Epoch 83/100
468/468 [==============================] - 0s 604us/step - loss: 0.6056 - accuracy: 0.6787
Epoch 84/100
468/468 [==============================] - 0s 602us/step - loss:

In [16]:
#Check the model's loss and accuracy 
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

156/156 - 0s - loss: 0.6122 - accuracy: 0.6807 - 152ms/epoch - 975us/step
Loss: 0.612212061882019, Accuracy: 0.6807144284248352


In [17]:
#Compare the test data to the predictions
predictions = nn_model.predict(X_test).reshape(4983).tolist()
pd.DataFrame({'y_test':y_test[0],'predictions':predictions}).sort_values(by=['predictions']).head(50)

156/156 [==============================] - 0s 469us/step


,y_test,predictions
17778,0,0.078026
17793,0,0.084267
17730,0,0.085115
17693,0,0.086307
17794,0,0.086913
17803,1,0.088575
17761,0,0.100584
17495,0,0.104842
17735,0,0.106123
17775,0,0.107542


In [ ]:
nn_model.save()